In [1]:
stock = 'BABA'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    187.039993
2018-07-24    189.000000
2018-07-25    197.979996
2018-07-26    194.179993
2018-07-27    189.419998
2018-07-30    184.820007
2018-07-31    187.229996
2018-08-01    185.270004
2018-08-02    182.600006
2018-08-03    180.839996
2018-08-06    178.619995
2018-08-07    179.919998
2018-08-08    177.520004
2018-08-09    177.190002
2018-08-10    180.009995
2018-08-13    177.679993
2018-08-14    172.529999
2018-08-15    169.830002
2018-08-16    171.990005
2018-08-17    172.779999
2018-08-20    176.289993
2018-08-21    177.919998
2018-08-22    177.850006
2018-08-23    172.229996
2018-08-24    174.229996
2018-08-27    180.649994
2018-08-28    178.190002
2018-08-29    178.500000
2018-08-30    174.600006
2018-08-31    175.009995
                 ...    
2018-09-18    156.649994
2018-09-19    162.630005
2018-09-20    165.880005
2018-09-21    164.630005
2018-09-24    163.160004
2018-09-25    164.250000
2018-09-26    165.399994
2018-09-27    166.320007
2018-09-28    164.75

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


142.6231246301667
[149.38169797 144.63589598 136.9040728  138.93914731 143.2548091 ]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")